<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Menu program

##Import Files:

In [2]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

Saving Board_Game_Info.txt to Board_Game_Info.txt
Saving Booking.txt to Booking.txt
Saving Rental.txt to Rental.txt
Saving feedbackManager.pyc to feedbackManager.pyc
Saving subscriptionManager.pyc to subscriptionManager.pyc
Saving Game_Feedback.txt to Game_Feedback.txt
Saving Subscription_Info.txt to Subscription_Info.txt
Saving Video_Game_Info.txt to Video_Game_Info.txt


#Register / login

Contains functions to:
- Check a user subscription type and username length
- Check if a user has registered
- Register a user using the subscriptionManager

Writes to the Subscription_Info.txt file with user information when a user registers

In [3]:

import ipywidgets as widgets
import subscriptionManager as smTV
import datetime
from dateutil.relativedelta import relativedelta as rd


def checkSubscription(userID):
    import subscriptionManager as smTV
    subscriptions = smTV.load_subscriptions()
    if smTV.check_subscription(userID, subscriptions):
        return True
    else:
        return False

def getSubscriptionType(userID):
    subscriptions = smTV.load_subscriptions()
    for item in subscriptions:
        if item == userID:
            data = subscriptions[item]
            return data["SubscriptionType"]
    return None

def register(userID, subscriptionType):


    subscriptions = smTV.load_subscriptions()

    if smTV.check_subscription(userID, subscriptions):
        with output:
            clear_output()
            print(f"{userID} is already registered")
        return

    if checkUserLength(userID) == False:
        with output:
            clear_output()
            print("Invalid user ID, must be 4 characters long")
        return


    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    with output:
        clear_output()
        print(f"{userID} has been registered")
    return

def checkUserLength(userID):
    if len(userID) != 4:
        return False
    else:
        return True

#Read and update text files:

##Update functions
Uses a dictionary and a list of headers to delete and rewrite text files

A different technique is used for the gameInfo files and the rental file.
- The program uses dictionaries when it comes to using gameInfo, so the csv.DictWriter is used to write to the files
- The rental system uses 2D lists so the file.write function is easier there. The ','.join(row) means each item in each row written is joined by a comma. The the file writer then moves down a line with a \n.

##getVideoDict() and getBoardDict()
Makes use of the csv package to read from the gameInfo files and makes a dictionary with the gameID as the key to each game.

- The {row["GameID"]: row for row in reader} loops through each row that the reader gives and sets the gameID to be the key for that row.

The function then re-calculates the available/unavailable numbers:
- Normalises the values by setting the available amount to (available + unavailable) and setting unavailable to 0, for each game
- Uses the rental file to check how many of a game has been rented and using that, re-calculates the available/unavailable amounts
- This is to keep the txt files in sychronisation



In [4]:
def updateVideo(videoDict):
    headers = ["GameID", "Game name", "Platform", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Video_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in videoDict.items():
        writer.writerow(value)
    return

def updateBoard(boardDict):
    headers = ["GameID", "Game name", "No of players", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Board_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in boardDict.items():
        writer.writerow(value)
    return

def updateRental(rentalList):
    with open("Rental.txt", mode="w", newline="") as file:
        file.write("Rental userID,GameID,Rental date,Return date\n")
        for row in rentalList:
            file.write(f"{','.join(row)}\n")
    return

def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        videoGameDict = {row["GameID"]: row for row in reader}
        for GameID in videoGameDict:
            gameData = videoGameDict[GameID]
            available = int(gameData["Available copies"])
            unavailable = int(gameData["Unavailable copies"])
            gameData["Available copies"] = str(available + unavailable)
            gameData["Unavailable copies"] = "0"

    with open("Rental.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["GameID"] in videoGameDict and row["Return date"] == "":
                videoGameDict[row["GameID"]]["Available copies"] = str(int(videoGameDict[row["GameID"]]["Available copies"]) - 1)
                videoGameDict[row["GameID"]]["Unavailable copies"] = str(int(videoGameDict[row["GameID"]]["Unavailable copies"]) + 1)

    updateVideo(videoGameDict)
    return videoGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
        for GameID in boardGameDict:
            gameData = boardGameDict[GameID]
            available = int(gameData["Available copies"])
            unavailable = int(gameData["Unavailable copies"])
            gameData["Available copies"] = str(available + unavailable)
            gameData["Unavailable copies"] = "0"

    with open("Rental.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["GameID"] in boardGameDict and row["Return date"] == "":
                boardGameDict[row["GameID"]]["Available copies"] = str(int(boardGameDict[row["GameID"]]["Available copies"]) - 1)
                boardGameDict[row["GameID"]]["Unavailable copies"] = str(int(boardGameDict[row["GameID"]]["Unavailable copies"]) + 1)

    updateBoard(boardGameDict)
    return boardGameDict



#Game search and print

##printGames()
Prints all the games with the specific platform and genre passed in as parameters.

##printGameAvailability()
Prints the availabilty of a specific game using its gameID.

##printGameInfo()
Prints all the infomation of a specific game and formats it nicely.


In [5]:
import csv


output = widgets.Output()


def printGames(platform, genre):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    with output:
        clear_output()
        print(f"Game ID: Game name:                          Type:          Available / Unavailable:\n")
        if platform == "All" and genre == "All":
            for i in boardDict:
                currentGame = boardDict[i]
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Board game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
            for i in videoDict:
                currentGame = videoDict[i]
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Video game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")

        for i in boardDict:
            currentGame = boardDict[i]
            if platform == "All" and genre in boardDict[i]["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Board game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        for i in videoDict:
            currentGame = videoDict[i]
            if platform in currentGame["Platform"] and genre in videoDict[i]["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Video game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
            elif platform == "All" and genre in currentGame["Genre"]:
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Video game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
            elif platform in currentGame["Platform"] and genre == "All":
                print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <35} Video game     {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")

def printGameAvailability(GameID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if GameID == "":
        return
    if GameID in boardDict:
        currentGame = boardDict[GameID]
        print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
    elif GameID in videoDict:
        currentGame = videoDict[GameID]
        print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
    else:
        return


def printGameInfo(GameID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()

    GameID = GameID.lower()
    if GameID == "":
        return
    if GameID not in boardDict and GameID not in videoDict:
        for key, value in boardDict.items():
            if GameID in value["Game name"].lower():
                GameID = key
                break
        for key, value in videoDict.items():
            if GameID in value["Game name"].lower():
                GameID = key
                break
    with output:
        clear_output()
        if GameID in boardDict:
            gameInfo = boardDict[GameID]
            for key, value in gameInfo.items():
                print(f"{key: >20}:  {value}")
            print("\n")
        elif GameID in videoDict:
            gameInfo = videoDict[GameID]
            for key, value in gameInfo.items():
                print(f"{key: >20}:  {value}")
            print("\n")
        else:
            return





#Game rent

Includes logic for renting games and updating the available/unavailable numbers for the rented or returned game.

Each user has a rental limit of 2 (basic subscription) or 7 (premium subscription), and the rentGameMain() function handles that.

The updateRentGame() function takes a user and a game, and adds the rental to the rental.txt file

In [6]:
import subscriptionManager as smTV
import feedbackManager as fmTV
from dateutil.relativedelta import relativedelta as rd
import datetime


def updateBoardFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    boardDict = getBoardDict()
    availableCopies = boardDict[GameID]["Available copies"]
    unavailableCopies = boardDict[GameID]["Unavailable copies"]
    boardDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    boardDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateBoard(boardDict)

def updateVideoFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    videoDict = getVideoDict()
    availableCopies = videoDict[GameID]["Available copies"]
    unavailableCopies = videoDict[GameID]["Unavailable copies"]
    videoDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    videoDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateVideo(videoDict)

def updateRentGame(gameID, userID):
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    with open("Rental.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{gameID},{startTime},")


def checkAvailability(gameID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameID in boardDict:
        currentGame = boardDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True

    elif gameID in videoDict:
        currentGame = videoDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True
    else:
        return None


def rentGameMain(gameID, userID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if not checkRentalLimit(userID):
        limit = smTV.get_rental_limit(getSubscriptionType(userID))
        with output:
            clear_output()
            print(f"Cannot rent game - User has reached their rental limit of {limit}")
        return
    if gameID in videoDict:
        updateVideoFromRentals(gameID, "rent")
        updateRentGame(gameID, userID)
        currentGame = videoDict[gameID]
        with output:
            print(f"{userID} has successfully rented {currentGame['Game name']}")
    elif gameID in boardDict:
        updateBoardFromRentals(gameID, "rent")
        updateRentGame(gameID, userID)
        currentGame = boardDict[gameID]
        with output:
            print(f"{userID} has successfully rented {currentGame['Game name']}")


def checkRentalLimit(userID):
    limit = 0

    subType = getSubscriptionType(userID)
    if subType == "Basic":
        limit = 2
    elif subType == "Premium":
        limit = 7
    else:
        return None

    rentalList = getRentalList()
    userRentals = getUserRental(userID, rentalList)
    if len(userRentals) >= limit:
        return False
    else:
        return True


#Returns and Feedback

##Returns

###getRentalList() and getUserRentals()
The rental system uses a 2D list to access data, it is read from rental.txt. userRentals is the list of games a specific user is renting.

###printRentedGames()
This is the function that is called by the searchRentedGames button in the UI, so handles the text output to guide the user.

It uses the userRentals to format and print all the games a specific user is currently renting.

###updateReturnList()
Updates the return list with a new rental: the userID and the gameID

###returnGame()
Calls the necessary functions to update the return list and the gameInfo dictionary and tells the user if the return was successfull

##Feedback
This section uses the feedbackManager package to add a review

In [7]:
import datetime
import feedbackManager as fmTV


def getRentalList():
    with open("Rental.txt", mode="r", newline="") as file:
        count = 0
        rentalList = []
        for line in file:
            if count == 0:
                count += 1
                continue
            else:
                currentLine = line.rstrip().split(",")
            count += 1
            rentalList.append(currentLine)
    return rentalList

def getUserRental(userID, rentalList):
    userRentals = []
    for row in rentalList:
        if row[0] == userID and row[3] == "":
            userRentals.append(row)
    return userRentals

def printRentedGames(userID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()

    date = datetime.datetime.now()
    rentalList = getRentalList()

    if userID == False:
        return
    if checkSubscription(userID) == False:
        with output:
            clear_output()
            print("User not registered")
            return

    userRental = getUserRental(userID, rentalList)
    if len(userRental) == 0:
        with output:
            print("No rented games")
        return
    with output:
        print("Rented games:")
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    for row in userRental:
        gameID = row[1]
        with output:
            if gameID in videoDict:
                print(f"{gameID}:  {videoDict[gameID]['Game name']}")
            elif gameID in boardDict:
                print(f"{gameID}:  {boardDict[gameID]['Game name']}")


def updateReturnList(rentalList, userID, gameID):
    for row in rentalList:
        if row[0] == userID and row[1] == gameID and row[3] == '':
            row[3] = datetime.datetime.now().strftime("%Y-%m-%d")
            break
    return rentalList


def returnGame(userID, gameID):
    videoDict = getVideoDict()
    boardDict = getBoardDict()

    rentalList = getRentalList()

    if gameID in videoDict:
        rentalList = updateReturnList(rentalList, userID, gameID)
        updateVideoFromRentals(gameID, "return")
        updateRental(rentalList)
        with output:
            print(f"{userID} has successfully returned {videoDict[gameID]['Game name']}")
            return True
    elif gameID in boardDict:
        rentalList = updateReturnList(rentalList, userID, gameID)
        updateBoardFromRentals(gameID, "return")
        updateRental(rentalList)
        with output:
            print(f"{userID} has successfully returned {boardDict[gameID]['Game name']}")
            return True
    else:
        with output:
            print("Invalid gameID")
        return False


# Feedback

def feedback(gameID, rating, comment):
    fmTV.add_feedback(gameID, rating, comment, "Game_Feedback.txt")
    return



# Booking
Contains all the logic to book a user onto a timeslot and check if they are eligable.

Makes use of the booking file to create a 2D booking list for every booking: userID, Time, Date, Number of guests.

Uses date objects to simplify the date logic to check whether the timeslot is full, and if the user has selected a date in the past.


In [8]:
def getBookingList():
    with open("Booking.txt", mode="r", newline="") as file:
        count = 0
        bookingList = []
        for line in file:
            if count == 0:
                count += 1
                continue
            else:
                currentLine = line.rstrip().split(",")
            count += 1
            bookingList.append(currentLine)
    return bookingList

def updateBooking(userID, date, time, guests):
    with open("Booking.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{date},{time},{guests}")
    return

def getBookedPeople(selectedDate, time, bookingList):
    bookedPeople = 0
    for row in bookingList:
        if row[1] == selectedDate.strftime("%Y-%m-%d") and row[2] == time:
            bookedPeople = bookedPeople + int(row[3]) + 1
    return bookedPeople


def bookingMain(userID, selectedDate, time, guests):
    bookingList = getBookingList()
    today = datetime.date.today()
    bookedPeople = getBookedPeople(selectedDate, time, bookingList)

    if bookedPeople + int(guests) + 1 > 50:
        with output:
            print("Not enough space in the selected timeslot")
            print(f"{bookedPeople} people have already booked this slot, 50 is the maximum we can hold")
        return
    if selectedDate < today:
        with output:
            print("Invalid date")
        return
    for item in bookingList:
        if item[0] == userID and item[1] == selectedDate and item[2] == time:
            with output:
                print("Timeslot already booked")
            return

    updateBooking(userID, selectedDate, time, guests)
    with output:
        print(f"Booking confirmed for {userID} on {selectedDate}")


# Pruning and graphs

##Logic for getting and using feedback
getBoardFeedbackList() and getVideoFeedbackList()

Uses the game feeback file to calculate each games rating and number of reviews.

Outputs the data in a list so it is easily accesible to all functions that use this data.

##Logic for plotting graphs
Uses the data from the list and plots it on bar graphs using matplotlib.

##Game pruning suggestions based on what the graphs show
To make good suggestions, it calculates the overall mean rating for both board and video games, and also finds the worst rated game for each.

If the worst rated game is less than 2 stars below the mean, it is suggested to be pruned - if not, no outliers exist and no suggestions are made.

##Pruning logic
Deletes the game from all gameInfo files so the game doesnt appear anywhere else again, but its past ratings and rental data exists, so if that infomation is needed at somepoint or if the game is added back into the system, the data still exists.


In [9]:
from typing_extensions import final
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

def getFeedback():
    try:
        fmTV.load_feedback()
    except StopIteration:
        return []
    return fmTV.load_feedback()

# headers = ["GameID", "Rating", "Comments"]
# writer = csv.DictWriter(open("Game_Feedback.txt", mode="w", newline=""), fieldnames=headers)
# writer.writeheader()



def getBoardFeedbackList():
    xAxis = []
    yAxisBar = []
    yAxisPie = []

    boardDict = getBoardDict()
    feedback = getFeedback()
    if feedback == []:
        return [[], [], []]

    gameRatings = {}
    for item in feedback:
        gameID = item["GameID"]
        if gameID in boardDict:
            if gameID not in gameRatings:
                gameRatings[gameID] = []
            gameRatings[gameID].append(item["Rating"])


    for gameID, ratingsList in gameRatings.items():
        averageRating = sum(ratingsList) / len(ratingsList)
        totalRatings = len(ratingsList)

        xAxis.append(gameID)
        yAxisBar.append(averageRating)
        yAxisPie.append(totalRatings)

    return [xAxis, yAxisBar, yAxisPie]

def getVideoFeedbackList():
    xAxis = []
    yAxisBar = []
    yAxisPie = []

    videoDict = getVideoDict()
    feedback = getFeedback()
    if feedback == []:
        return [[], [], []]

    gameRatings = {}
    for item in feedback:
        gameID = item["GameID"]
        if gameID in videoDict:
            if gameID not in gameRatings:
                gameRatings[gameID] = []
            gameRatings[gameID].append(item["Rating"])


    for gameID, ratingsList in gameRatings.items():
        averageRating = sum(ratingsList) / len(ratingsList)
        totalRatings = len(ratingsList)

        xAxis.append(gameID)
        yAxisBar.append(averageRating)
        yAxisPie.append(totalRatings)

    return [xAxis, yAxisBar, yAxisPie]

def plotVideoAvgChart():
    x, y, unused = getVideoFeedbackList()
    plt.bar(np.array(x), np.array(y))
    plt.xlabel("GameID")
    plt.ylabel("Average Rating")
    plt.title("Video Game Average Feedback")
    plt.xticks(rotation=90, ha="right")
    plt.xticks(fontsize=6)
    with output:
        plt.show()

def plotVideoNumChart():
    x, unused, y = getVideoFeedbackList()
    plt.bar(np.array(x), np.array(y))
    plt.xlabel("GameID")
    plt.ylabel("Number of reviews")
    plt.title("Video Game Number of Reviews")
    plt.xticks(rotation=90, ha="right")
    plt.xticks(fontsize=6)
    with output:
        plt.show()

def plotBoardAvgChart():
    x, y, unused = getBoardFeedbackList()
    plt.bar(np.array(x), np.array(y))
    plt.xlabel("GameID")
    plt.ylabel("Average Rating")
    plt.title("Board Game Average Feedback")
    plt.xticks(rotation=90, ha="right")
    plt.xticks(fontsize=6)
    with output:
        plt.show()

def plotBoardNumChart():
    x, unused, y = getBoardFeedbackList()
    plt.bar(np.array(x), np.array(y))
    plt.xlabel("GameID")
    plt.ylabel("Number of reviews")
    plt.title("Board Game Number of Reviews")
    plt.xticks(rotation=90, ha="right")
    plt.xticks(fontsize=6)
    with output:
        plt.show()


def pruneGame(gameID):
    boardDict = getBoardDict()
    videoDict = getVideoDict()
    if gameID in boardDict:
        boardDict.pop(gameID)
        updateBoard(boardDict)
        return True

    elif gameID in videoDict:
        videoDict.pop(gameID)
        updateVideo(videoDict)
        return True

    else:
        return False

def getPruningSuggestions():
    boardDict = getBoardDict()
    videoDict = getVideoDict()
    boardFeeback = getBoardFeedbackList()
    videoFeedback = getVideoFeedbackList()

    worstBoardRating = 5
    for item in range(len(boardFeeback[1])):
        if boardFeeback[1][item] < worstBoardRating:
            worstBoardRating = boardFeeback[1][item]
            worstRatedBoard = boardFeeback[0][item]

    worstVideoRating = 5
    for item in range(len(videoFeedback[1])):
        if videoFeedback[1][item] < worstVideoRating:
            worstVideoRating = videoFeedback[1][item]
            worstRatedVideo = videoFeedback[0][item]

    meanBoardRating = 0
    for item in range(len(boardFeeback[1])):
        meanBoardRating += boardFeeback[1][item]
    if len(boardFeeback[1]) == 0:
        meanBoardRating = 0
    else:
        meanBoardRating = meanBoardRating / len(boardFeeback[1])

    meanVideoRating = 0
    for item in range(len(videoFeedback[1])):
        meanVideoRating += videoFeedback[1][item]
    if len(videoFeedback[1]) == 0:
        meanVideoRating = 0
    else:
        meanVideoRating = meanVideoRating / len(videoFeedback[1])

    if meanBoardRating - worstBoardRating < 2:
        boardSuggestions = None
        boardBool = True
    else:
        boardSuggestions = [worstRatedBoard, worstBoardRating]
        boardBool = False

    if meanVideoRating - worstVideoRating < 2:
        videoSuggestions = None
        videoBool = True
    else:
        videoSuggestions = [worstRatedVideo, worstVideoRating]
        videoBool = False

    return [boardBool, boardSuggestions, videoBool, videoSuggestions]


def printPruningSuggestions(suggestions):
    boardDict = getBoardDict()
    videoDict = getVideoDict()
    with output:
        clear_output()
        print("-- Suggested games to delete --")

    if suggestions[0] == True:
        with output:
            print("Board games: No suggestions, there are no extreme outliers.")
    else:
        game = suggestions[1][0]
        gameData = boardDict[game]
        gameName = gameData["Game name"]
        with output:
            print(f"Board games: {suggestions[1][0]} ({gameName}). It has the lowest rating, {round(suggestions[1][1], 2)}/5")

    if suggestions[2] == True:
        with output:
            print("Video games: No suggestions, there are no extreme outliers.")
    else:
        game = suggestions[3][0]
        gameData = videoDict[game]
        gameName = gameData["Game name"]
        with output:
            print(f"Video games: {suggestions[3][0]} ({gameName}). It has the lowest rating, {round(suggestions[3][1], 2)}/5")


# Widgets, Buttons and UI

This cell contains:
- The functions that are called when any button is clicked
- The ipywidgets code to create, hide, and show widgets, and the logic behind them to create a smooth user interface
- The initialisation of the Game Store Menu program - run this cell to open the UI




In [10]:
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import clear_output


def resetValues():
    enterGameID.value = ""
    enterUserID.value = ""
    dropdownPlatform.value = "All"
    dropdownGenre.value = "All"
    timeslotDropdown.value = "2pm"
    guestsDropdown.value = "1"
    timeWidget.value = None
    accountDropdown.value = "Basic"
    feedbackDropdown.value = "5"
    feedbackText.value = ""


def on_search_clicked(b):
    resetValues()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    rentInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = 'none'

    searchInputBox.layout.display = ''

    ioBox.layout.display = ''
    with output:
        clear_output()
        print("Enter gameID/Game name, or use the dropdown menus to search")


def on_rent_clicked(b):
    resetValues()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    searchInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = 'none'
    rentInputBox.layout.display = ''

    ioBox.layout.display = ''
    with output:
        clear_output()

def on_return_clicked(b):
    resetValues()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    searchInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    rentInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = 'none'
    returnInputBox.layout.display = ''

    ioBox.layout.display = ''



def on_book_clicked(b):
    resetValues()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    searchInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    rentInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = 'none'
    bookInputBox.layout.display = ''

    ioBox.layout.display = ''


def on_register_clicked(b):
    resetValues()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    rentInputBox.layout.display = 'none'
    searchInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = 'none'
    registerInputBox.layout.display = ''


    ioBox.layout.display = ''
    with output:
        clear_output()


def on_feedback_section_clicked(b):
    resetValues()
    with output:
        clear_output()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    rentInputBox.layout.display = 'none'
    searchInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'

    pruningSectionButton.layout.display = ''
    feedbackSectionButton.layout.display = 'none'

    feedbackChartsBox.layout.display = ''
    pruningBox.layout.display = 'none'

    ioBox.layout.display = ''


def on_pruning_section_clicked(b):
    resetValues()
    with output:
        clear_output()
    menuBox.layout.display = 'none'
    quitBox.layout.display = ''

    rentInputBox.layout.display = 'none'
    searchInputBox.layout.display = 'none'
    returnInputBox.layout.display = 'none'
    bookInputBox.layout.display = 'none'
    registerInputBox.layout.display = 'none'
    feedbackBox.layout.display = 'none'
    feedbackSectionButton.layout.display = ''
    pruningSectionButton.layout.display = 'none'

    feedbackChartsBox.layout.display = 'none'
    pruningBox.layout.display = ''

    ioBox.layout.display = ''


def main_menu(b):
    menuBox.layout.display = ''
    quitBox.layout.display = 'none'
    feedbackSectionButton.layout.display = 'none'
    pruningSectionButton.layout.display = 'none'
    ioBox.layout.display = 'none'
    with output:
        clear_output()

def on_search_for_game_clicked(b):
    # If the gameID textbox is left empty, all games are
    # searched for using the parameters

    # If not, it doesnt matter what genre/platform is given, it will print out
    # info for only that game
    if enterGameID.value == "":
        printGames(dropdownPlatform.value, dropdownGenre.value)
    else:
        printGameInfo(enterGameID.value)
        dropdownPlatform.value = "All"
        dropdownGenre.value = "All"

def on_checkAvailability_clicked(b):
    if checkAvailability(enterGameID.value) == None:
        with output:
            clear_output()
            print("Invalid GameID")
    elif checkAvailability(enterGameID.value) == False:
        with output:
            clear_output()
            printGameAvailability(enterGameID.value)
            print("Game not available")
    else:
        with output:
            clear_output()
            printGameAvailability(enterGameID.value)
            print("Game available")

def rent_button_clicked(b):
    if checkSubscription(enterUserID.value) == False:
        with output:
            clear_output()
            print("User not registered")
            return
    elif checkAvailability(enterGameID.value) == None:
        with output:
            clear_output()
            print("Invalid gameID")
            return
    elif checkAvailability(enterGameID.value) == False:
        with output:
            clear_output()
            print("Game not available")
            return
    else:
        rentGameMain(enterGameID.value, enterUserID.value)
    resetValues()


def on_registerUser_clicked(b):
    register(enterUserID.value, accountDropdown.value)
    resetValues()

def on_returnGame_clicked(b):
    with output:
        clear_output()
    if returnGame(enterUserID.value, enterGameID.value):
        returnInputBox.layout.display = 'none'
        feedbackBox.layout.display = ''
        with output:
            print("Please enter some feedback for this game")
        return


def on_searchRented_clicked(b):
    with output:
        clear_output()
    if checkSubscription(enterUserID.value) == False:
        with output:
            clear_output()
            print("User not registered")
            return
    printRentedGames(enterUserID.value)

def on_bookUser_click(b):
    with output:
        clear_output()
    if checkSubscription(enterUserID.value) == False:
        with output:
            clear_output()
            print("User not registered")
            return
    if timeWidget.value == None:
        with output:
            clear_output()
            print("Please enter a date")
            return
    bookingMain(enterUserID.value, timeWidget.value, timeslotDropdown.value, guestsDropdown.value)
    resetValues()

def onFeedbackClicked(b):
    gameID = enterGameID.value
    if feedbackText.value == "":
        with output:
            clear_output()
            print("Please enter your feedback")
            return

    feedback(gameID, int(feedbackDropdown.value), feedbackText.value)
    resetValues()
    feedbackBox.layout.display = 'none'
    returnInputBox.layout.display = ''
    with output:
        clear_output()
        print("Thank you for your feedback")

def on_prune_game_clicked(b):
    with output:
        clear_output()
    if not pruneGame(enterGameID.value):
        with output:
            clear_output()
            print("Game not found")
    else:
        with output:
            clear_output()
            print("Game pruned from gameInfo files")
    resetValues()

def on_pruning_suggestions_clicked(b):
    with output:
        clear_output()
    pruningSuggestions = getPruningSuggestions()
    printPruningSuggestions(pruningSuggestions)

def on_board_avg_clicked(b):
    with output:
        clear_output()
    plotBoardAvgChart()

def on_board_num_clicked(b):
    with output:
        clear_output()
    plotBoardNumChart()

def on_video_avg_clicked(b):
    with output:
        clear_output()
    plotVideoAvgChart()

def on_video_num_clicked(b):
    with output:
        clear_output()
    plotVideoNumChart()



    # Menu widgets

menuButtonSearch = widgets.Button(description="Search games", button_style="primary")
menuButtonSearch.on_click(on_search_clicked)

menuButtonRent = widgets.Button(description="Rent a game", button_style="primary")
menuButtonRent.on_click(on_rent_clicked)

menuButtonReturn = widgets.Button(description="Return a game", button_style="primary")
menuButtonReturn.on_click(on_return_clicked)

menuButtonBook = widgets.Button(description="Book a timeslot", button_style="primary")
menuButtonBook.on_click(on_book_clicked)

menuButtonRegister = widgets.Button(description="Register user", button_style="primary")
menuButtonRegister.on_click(on_register_clicked)

menuButtonPruning = widgets.Button(description="Pruning + Feedback", button_style="primary")
menuButtonPruning.on_click(on_pruning_section_clicked)

quitButton = widgets.Button(description="Quit")
quitButton.on_click(main_menu)

paddingBox = widgets.VBox([], layout=widgets.Layout(width='10px'))

enterUserID = widgets.Text(placeholder="Enter userID", layout=widgets.Layout(width='200px'))


    # Game search widgets

enterGameID = widgets.Text(placeholder="Enter GameID", layout=widgets.Layout(width='200px'))

dropdownPlatform = widgets.Dropdown(options=["All", "PlayStation", "Xbox", "PC", "Nintendo"],
                                    value = "All",
                                    description="Platform:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

dropdownGenre = widgets.Dropdown(options=["All", "Action-Adventure", "RPG", "Shooter", "Simulation", "Sandbox",
                                    "Action", "Platformer", "Racing", "Strategy", "Mystery", "Family", "Quiz", "Word"],
                                    value = "All",
                                    description="Genre:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

buttonSearchForGame = widgets.Button(description="Search", layout=widgets.Layout(width='120px'))
buttonSearchForGame.on_click(on_search_for_game_clicked)


searchInputBox = widgets.HBox([enterGameID, dropdownPlatform, dropdownGenre, buttonSearchForGame])



    # Rent game widgets

checkAvailabilityButton = widgets.Button(description="Check availability")
checkAvailabilityButton.on_click(on_checkAvailability_clicked)

rentGameButton = widgets.Button(description="Rent game")
rentGameButton.on_click(rent_button_clicked)

rentInputBox = widgets.HBox([enterUserID, enterGameID, checkAvailabilityButton, rentGameButton])



    # Return game widgets

returnGameButton = widgets.Button(description="Return game")
returnGameButton.on_click(on_returnGame_clicked)

searchRentedGamesButton = widgets.Button(description="Search rented games")
searchRentedGamesButton.on_click(on_searchRented_clicked)

returnInputBox = widgets.HBox([enterUserID, searchRentedGamesButton, enterGameID, returnGameButton])

feedbackDropdown = widgets.Dropdown(options=["0", "1", "2", "3", "4", "5"],
                                    value="5",
                                    description="Rating:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

feedbackText = widgets.Text(placeholder="Enter feedback", layout=widgets.Layout(width='200px'))

feedbackButton = widgets.Button(description="Submit")
feedbackButton.on_click(onFeedbackClicked)

feedbackBox = widgets.HBox([feedbackDropdown, feedbackText, feedbackButton])


    # Booking widgets

timeslotDropdown = widgets.Dropdown(options=["2pm", "6pm"],
                                    value="2pm",
                                    description="Time:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

guestsDropdown = widgets.Dropdown(options=["0", "1", "2", "3"],
                                    value="0",
                                    description="No of Guests:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

timeWidget = widgets.DatePicker(description='Date',
                                style={'description_width': 'initial'})

bookUserButton = widgets.Button(description="Book")
bookUserButton.on_click(on_bookUser_click)

bookInputBox = widgets.HBox([enterUserID, timeWidget, timeslotDropdown, guestsDropdown, bookUserButton])


    # Register user widgets

registerUserButton = widgets.Button(description="Register")
registerUserButton.on_click(on_registerUser_clicked)

accountDropdown = widgets.Dropdown(options=["Basic", "Premium"],
                                    value = "Basic",
                                    description="Account type:",
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='200px'))

registerInputBox = widgets.HBox([enterUserID, accountDropdown, registerUserButton])

    # Pruning widgets

feedbackSectionButton = widgets.Button(description="Feedback section", button_style="primary")
feedbackSectionButton.on_click(on_feedback_section_clicked)

pruningSectionButton = widgets.Button(description="Pruning section", button_style="primary")
pruningSectionButton.on_click(on_pruning_section_clicked)

boardChartLabel = widgets.Label(value=" Board games:")
videoChartLabel = widgets.Label(value=" Video games:")

boardAvgButton = widgets.Button(description="Average ratings")
boardAvgButton.on_click(on_board_avg_clicked)

boardNumButton = widgets.Button(description="Num of ratings")
boardNumButton.on_click(on_board_num_clicked)

videoAvgButton = widgets.Button(description="Average ratings")
videoAvgButton.on_click(on_video_avg_clicked)

videoNumButton = widgets.Button(description="Num of ratings")
videoNumButton.on_click(on_video_num_clicked)

pruningSuggestionsButton = widgets.Button(description="Pruning suggestions")
pruningSuggestionsButton.on_click(on_pruning_suggestions_clicked)

pruneGameButton = widgets.Button(description="Prune game")
pruneGameButton.on_click(on_prune_game_clicked)

pieChartBox = widgets.VBox([boardAvgButton, boardNumButton])
barChartBox = widgets.VBox([videoAvgButton, videoNumButton])

feedbackChartsBox = widgets.HBox([boardChartLabel, pieChartBox, videoChartLabel, barChartBox, pruningSuggestionsButton])

pruningBox = widgets.HBox([enterGameID, pruneGameButton])


    # I/O boxes

output = widgets.Output()

menuBox = widgets.VBox([menuButtonSearch, menuButtonRent, menuButtonReturn, menuButtonBook, menuButtonRegister, menuButtonPruning])
quitBox = widgets.VBox([quitButton, feedbackSectionButton, pruningSectionButton])

quitBox.layout.display = 'none'

ioBox = widgets.HBox([quitBox, widgets.VBox([searchInputBox, rentInputBox, returnInputBox, bookInputBox, registerInputBox, feedbackBox, feedbackChartsBox, pruningBox, output])])

mainBox = widgets.HBox([menuBox, paddingBox, ioBox])

display(mainBox)
main_menu('b')


